In [ ]:
# !pip3 install nltk==3.2.4
# !wget http://nlp.stanford.edu/software/stanford-ner-2015-04-20.zip
# !unzip stanford-ner-2015-04-20.zip

In [144]:
import json
import jsonlines
import pandas as pd

## Using NLTK
For playing around with it to familiarize oneself.

In [29]:

from nltk.tag.stanford import StanfordNERTagger
jar = "stanford-ner-2015-04-20/stanford-ner-3.5.2.jar"
model = "stanford-ner-2015-04-20/classifiers/" 

st_3class = StanfordNERTagger(model + "english.all.3class.distsim.crf.ser.gz", jar, encoding='utf8') 
st_4class = StanfordNERTagger(model + "english.conll.4class.distsim.crf.ser.gz", jar, encoding='utf8') 
st_7class = StanfordNERTagger(model + "english.muc.7class.distsim.crf.ser.gz", jar, encoding='utf8')

In [94]:
test_fn =  "../floresp-v2.0-rc.3/devtest/devtest.eng_Latn"
# test_fn = "hello.txt"
with open(test_fn, "r") as f:
    devtest_en = [line.strip() for line in f.readlines()]

In [38]:
for line in devtest_en:
    print(st_7class.tag(line.split()))

[('These', 'O'), ('are', 'O'), ('28', 'O'), ('numbers', 'O'), ('in', 'O'), ('1996', 'DATE'), ('that', 'O'), ('had', 'O'), ('99%.', 'O')]
[('I', 'O'), ('am', 'O'), ('doing', 'O'), ('interesting', 'O'), ('three', 'O'), ('things.', 'O'), ('But', 'O'), ('I', 'O'), ('have', 'O'), ('to', 'O'), ('leave', 'O'), ('at', 'O'), ('9:00', 'TIME'), ('am.', 'TIME')]
[('I', 'O'), ('went', 'O'), ('there', 'O'), ('for', 'O'), ('2', 'O'), ('weeks.', 'O')]
[('She', 'O'), ('drove', 'O'), ('for', 'O'), ('92', 'O'), ('km.', 'O')]


# Using Stanford CoreNLP to run
Downloaod [CoreNLP](https://stanfordnlp.github.io/CoreNLP/) (Java) and run it.

Input: The English text file to annotate

Output: A JSON file of lines with numerical text

In [127]:
test_fn =  "../floresp-v2.0-rc.3/devtest/devtest.eng_Latn"
with open(test_fn, "r") as f:
    devtest_en = [line.strip() for line in f.readlines()]

In [41]:
!pwd

/Users/ogayo/dev/wmt_2024_main/wmt_24_work


In [66]:
!cd  ../stanford-corenlp-4.5.7/ && java -cp "*" edu.stanford.nlp.pipeline.StanfordCoreNLP -annotators tokenize,pos,lemma,ner -ner.rulesOnly -file "../floresp-v2.0-rc.3/devtest/devtest.eng_Latn" -outputFormat json -outputDirectory annotation_output -ssplit.eolonly

[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator tokenize
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator pos
[main] INFO edu.stanford.nlp.tagger.maxent.MaxentTagger - Loading POS tagger from edu/stanford/nlp/models/pos-tagger/english-left3words-distsim.tagger ... done [0.6 sec].
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator lemma
[main] INFO edu.stanford.nlp.pipeline.StanfordCoreNLP - Adding annotator ner
[main] INFO edu.stanford.nlp.time.JollyDayHolidays - Initializing JollyDayHoliday for SUTime from classpath edu/stanford/nlp/models/sutime/jollyday/Holidays_sutime.xml as sutime.binder.1.
[main] INFO edu.stanford.nlp.time.TimeExpressionExtractorImpl - Using following SUTime rules: edu/stanford/nlp/models/sutime/defs.sutime.txt,edu/stanford/nlp/models/sutime/english.sutime.txt,edu/stanford/nlp/models/sutime/english.holidays.sutime.txt
[main] INFO edu.stanford.nlp.pipeline.TokensRegexNERAnnotator - ner.fine.reg

In [67]:
output_fn = "/Users/ogayo/dev/wmt_2024_main/stanford-corenlp-4.5.7/annotation_output/devtest.eng_Latn.json"

In [68]:
with open(output_fn, "r") as f:
    data = json.load(f)

In [69]:
len(data)

2

In [70]:
data.keys()

dict_keys(['docId', 'sentences'])

In [71]:
len(data['sentences'])

1012

In [75]:
data['sentences'][0]['entitymentions']

[{'docTokenBegin': 2,
  'docTokenEnd': 3,
  'tokenBegin': 2,
  'tokenEnd': 3,
  'text': 'now',
  'characterOffsetBegin': 4,
  'characterOffsetEnd': 7,
  'ner': 'DATE',
  'normalizedNER': 'PRESENT_REF',
  'nerConfidences': {'DATE': -1},
  'timex': {'tid': 't1', 'type': 'DATE', 'value': 'PRESENT_REF'}},
 {'docTokenBegin': 4,
  'docTokenEnd': 9,
  'tokenBegin': 4,
  'tokenEnd': 9,
  'text': '4-month-old',
  'characterOffsetBegin': 13,
  'characterOffsetEnd': 24,
  'ner': 'DURATION',
  'normalizedNER': 'P4M',
  'nerConfidences': {'DURATION': -1},
  'timex': {'tid': 't2', 'type': 'DURATION', 'value': 'P4M'}},
 {'docTokenBegin': 20,
  'docTokenEnd': 21,
  'tokenBegin': 20,
  'tokenEnd': 21,
  'text': 'he',
  'characterOffsetBegin': 79,
  'characterOffsetEnd': 81,
  'ner': 'PERSON'}]

In [118]:
"""
source = https://stanfordnlp.github.io/CoreNLP/ner.html#description
For English, by default, this annotator recognizes named (PERSON, LOCATION, ORGANIZATION, MISC), 
numerical (MONEY, NUMBER, ORDINAL, PERCENT), and temporal (DATE, TIME, DURATION, SET) entities (12 classes).
"""
numerical_categories = ["MONEY", "NUMBER", "ORDINAL", "PERCENT", "DATE", "TIME", "DURATION"]

In [128]:
numerical_lines_dict = {}
line_number_categories_dict = [] # To create a jsonline where each line will be {"MONEY":True, "NUMBER":False, "ORDINAL", etc}
for sent in data['sentences']:
    for entity  in sent['entitymentions']:
        default_line_dict = {"INDEX": None,"TEXT":None, "MONEY": False, "NUMBER":False, "ORDINAL":False, "PERCENT":False, "DATE":False, "TIME":False, "DURATION":False}
        if entity['ner'] in numerical_categories:
            if numerical_lines_dict.get(sent['index']):
                numerical_lines_dict[sent['index']] = numerical_lines_dict.get(sent['index'], 0) + 1
                default_line_dict[entity['ner']] =  default_line_dict.get(entity['ner']) + 1  if default_line_dict[entity['ner']] else 1
            else:
                numerical_lines_dict[sent['index']] = 1
                default_line_dict["INDEX"] = sent['index']
                default_line_dict["TEXT"] = devtest_en[sent['index']]
                default_line_dict[entity['ner']] = 1
                line_number_categories_dict.append(default_line_dict)


In [129]:
len(line_number_categories_dict)

417

In [130]:
line_number_categories_dict[1], devtest_en[1]

({'INDEX': 1,
  'TEXT': 'Dr. Ehud Ur, professor of medicine at Dalhousie University in Halifax, Nova Scotia and chair of the clinical and scientific division of the Canadian Diabetes Association cautioned that the research is still in its early days.',
  'MONEY': False,
  'NUMBER': False,
  'ORDINAL': False,
  'PERCENT': False,
  'DATE': False,
  'TIME': False,
  'DURATION': 1},
 'Dr. Ehud Ur, professor of medicine at Dalhousie University in Halifax, Nova Scotia and chair of the clinical and scientific division of the Canadian Diabetes Association cautioned that the research is still in its early days.')

In [131]:
len(numerical_lines_dict)

417

In [132]:
numerical_lines_dict.keys()

dict_keys([0, 1, 2, 3, 4, 5, 7, 8, 10, 11, 12, 18, 19, 23, 24, 27, 33, 34, 41, 42, 43, 44, 46, 48, 50, 51, 53, 54, 55, 56, 59, 60, 63, 64, 65, 66, 67, 68, 70, 71, 73, 74, 77, 80, 82, 83, 84, 85, 86, 87, 99, 100, 101, 102, 107, 108, 109, 110, 111, 112, 115, 116, 117, 118, 119, 120, 121, 123, 124, 132, 134, 136, 137, 139, 141, 144, 152, 153, 157, 158, 159, 160, 165, 166, 167, 168, 170, 171, 173, 174, 177, 178, 179, 180, 181, 183, 184, 185, 191, 193, 194, 196, 199, 200, 201, 202, 203, 204, 205, 206, 213, 214, 215, 221, 222, 223, 225, 226, 227, 228, 230, 232, 234, 236, 237, 244, 247, 248, 250, 251, 253, 254, 255, 257, 261, 265, 266, 268, 269, 271, 273, 274, 275, 276, 278, 281, 291, 292, 293, 296, 297, 298, 299, 300, 301, 302, 303, 304, 306, 311, 312, 315, 318, 320, 322, 323, 324, 325, 326, 328, 329, 332, 334, 339, 340, 347, 348, 356, 363, 366, 367, 371, 376, 382, 387, 399, 403, 404, 407, 408, 409, 414, 418, 420, 421, 423, 424, 425, 426, 427, 429, 430, 431, 432, 433, 438, 440, 441, 443, 446

In [133]:
"""
So the rule based duration tagger tags things like "early" as duration.
So I ran one excluding duration (393 lines)and one with duration (417 lines). 
The extra 24 lines should be mannually inspected to see if they fit
Also date needs to be worked on. Core NLP will select seasons such as winter as DATE.

"""
# without_duration_only = set(numerical_lines_dict.keys())
duration_only = set(numerical_lines_dict.keys()) - set(without_duration_only)
len(duration_only)

24

In [135]:
print(duration_only)

{1, 387, 783, 914, 409, 546, 931, 806, 170, 811, 690, 947, 821, 438, 441, 443, 70, 861, 738, 748, 494, 111, 244, 630}


In [136]:
line_number_categories_dict[0]

{'INDEX': 0,
 'TEXT': '"We now have 4-month-old mice that are non-diabetic that used to be diabetic," he added.',
 'MONEY': False,
 'NUMBER': False,
 'ORDINAL': False,
 'PERCENT': False,
 'DATE': 1,
 'TIME': False,
 'DURATION': False}

In [150]:
final_output_fn = "flores_with_numbers"

In [151]:
with open(final_output_fn+".json", 'w') as f:
    for dictionary in line_number_categories_dict:
        f.write(json.dumps(dictionary)+'\n')

In [152]:
df = pd.read_json(final_output_fn+".json", lines=True, orient='records')
df.head()

,INDEX,TEXT,MONEY,NUMBER,ORDINAL,PERCENT,DATE,TIME,DURATION
0,0,"""We now have 4-month-old mice that are non-dia...",0,0,0,0,1,0,0
1,1,"Dr. Ehud Ur, professor of medicine at Dalhousi...",0,0,0,0,0,0,1
2,2,"Like some other experts, he is skeptical about...",0,1,0,0,0,0,0
3,3,"On Monday, Sara Danius, permanent secretary of...",0,0,0,0,1,0,0
4,4,"Danius said, ""Right now we are doing nothing. ...",0,0,0,0,1,0,0


In [153]:
df.to_csv(final_output_fn+".csv", index=False)